In [1]:
import numpy as np
import pandas as pd
from sympy import diff, lambdify, symbols, log, sqrt
from sympy.stats import Normal, density
from scipy.stats import norm

kappa = 0.1465
alpha = 0.5172
omega = 0.5786
rho = -0.0243
v0 = 0.5172
r = 0
T = 1 / 12
K = 1000
xi = 0.5
# symbolic variables
s, v, t = symbols('s v t')
sigma = sqrt(v0)
N = Normal('x', 0, 1)
d_p = (log(s / K) + (r + v0 / 2) * t) / (sigma * sqrt(t))
D0 = 0.5 * (v - v0) * s * density(N)(d_p) / (sigma * sqrt(t))
delta0 = diff(D0, s)
gamma0 = diff(delta0, s)
vega0 = diff(D0, v)

S = [950+10*i for i in range(11)]

def delta(S, K, T, r, sigma):
  N = norm.cdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return N(d_p)

def gamma(S, K, T, r, sigma):
  N = norm.pdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return N(d_p) / (sigma * np.sqrt(T) * S)

def vega(S, K, T, r, sigma):
  N = norm.pdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return S * N(d_p) * np.sqrt(T) / (2 * sigma)

def L(f): # infinitesimal generator
  ft = diff(f, t)
  fs = diff(f, s)
  fs2 = diff(fs, s)
  fv = diff(f, v)
  fv2 = diff(fv, v)
  fvs = diff(fv, s)
  tmp1 = -ft + r * s * fs + kappa * (alpha - v) * fv
  tmp2 = (v * s**2 * fs2 + omega**2 * v**(2 * xi) * fv2) / 2
  tmp3 = rho * omega * v**(xi + 0.5) * s * fvs
  return tmp1 + tmp2 + tmp3 - r * f

D = [D0]
for _ in range(1, 5):
  D.append(L(D[-1]))

In [2]:
Delta = [delta0]
Delta_func = []
for i in range(1, 5):
  Delta.append(diff(D[i], s))
for i in range(0, 5):
  Delta_func.append(lambdify([s, v, t], Delta[i]))

Gamma = [gamma0]
Gamma_func = []
for i in range(1, 5):
  Gamma.append(diff(Delta[i], s))
for i in range(0, 5):
  Gamma_func.append(lambdify([s, v, t], Gamma[i]))

Vega = [vega0]
Vega_func = []
for i in range(1, 5):
  Vega.append(diff(D[i], v))
for i in range(0, 5):
  Vega_func.append(lambdify([s, v, t], Vega[i]))

In [3]:
%%time
Approx_delta = []
for s in S:
  delta0_approx = delta(s, K, T, r, np.sqrt(v0)) + T * Delta_func[0](s, v0, T)
  delta1_approx = delta0_approx + T**2 / 2 * Delta_func[1](s, v0, T)
  delta2_approx = delta1_approx + T**3 / 6 * Delta_func[2](s, v0, T)
  delta3_approx = delta2_approx + T**4 / 24 * Delta_func[3](s, v0, T)
  delta4_approx = delta3_approx + T**5 / 120 * Delta_func[4](s, v0, T)
  Approx_delta.append(100*delta4_approx)
print(Approx_delta)

[44.28194197068317, 46.29395968905791, 48.29452443499121, 50.27876292217998, 52.24207318867641, 54.18014586712322, 56.08898091174347, 57.96489994498629, 59.80455444024395, 61.604930000468926, 63.36334702619561]
Wall time: 4.38 s


In [4]:
%%time
Approx_gamma = []
for s in S:
  gamma0_approx = gamma(s, K, T, r, np.sqrt(v0)) + T * Gamma_func[0](s, v0, T)
  gamma1_approx = gamma0_approx + T**2 / 2 * Gamma_func[1](s, v0, T)
  gamma2_approx = gamma1_approx + T**3 / 6 * Gamma_func[2](s, v0, T)
  gamma3_approx = gamma2_approx + T**4 / 24 * Gamma_func[3](s, v0, T)
  gamma4_approx = gamma3_approx + T**5 / 120 * Gamma_func[4](s, v0, T)
  Approx_gamma.append(100*gamma4_approx)
print(Approx_gamma)

[0.20160569714389376, 0.2007124839166706, 0.19931920065028394, 0.19745176199529366, 0.1951384406755816, 0.19240940705835285, 0.18929628199757056, 0.18583170881341948, 0.18204894924999748, 0.17798150725804376, 0.1736627834998477]
Wall time: 9.01 s


In [5]:
%%time
Approx_vega = []
for s in S:
  vega0_approx = vega(s, K, T, r, np.sqrt(v0))
  vega1_approx = vega0_approx + T**2 / 2 * Vega_func[1](s, v0, T)
  vega2_approx = vega1_approx + T**3 / 6 * Vega_func[2](s, v0, T)
  vega3_approx = vega2_approx + T**4 / 24 * Vega_func[3](s, v0, T)
  vega4_approx = vega3_approx + T**5 / 120 * Vega_func[4](s, v0, T)
  Approx_vega.append(vega4_approx)
print(Approx_vega)

[74.96794881327439, 76.22123117968029, 77.28470529259899, 78.15627941528598, 78.83538891870604, 79.32292722222938, 79.62116243550538, 79.73364212769302, 79.6650886388748, 79.42128728789046, 79.00896972842897]
Wall time: 3.31 s


In [6]:
Delta_FFT = np.array([44.2794, 46.2918, 48.2928, 50.2776, 52.2414, 54.1800, 56.0893, 57.9657, 59.8058, 61.6066, 63.3654])
Delta_diff = (Approx_delta - Delta_FFT) / Delta_FFT * 100
Gamma_FFT = np.array([0.2016, 0.2008, 0.1994, 0.1975, 0.1952, 0.1925, 0.1893, 0.1859, 0.1821, 0.1780, 0.1737])
Gamma_diff = (Approx_gamma - Gamma_FFT) / Gamma_FFT * 100
Vega_FFT = np.array([74.9945, 76.2501, 77.3079, 78.1875, 78.8766, 79.3500, 79.6521, 79.7443, 79.6526, 79.4439, 79.0269])
Vega_diff = (Approx_vega - Vega_FFT) / Vega_FFT * 100

df = pd.DataFrame({
    "S": S, 
    "FFT %delta": Delta_FFT, 
    "Replicate %delta": np.round(Approx_delta, 4), 
    "Delta %diff": np.round(Delta_diff, 4),
    "FFT %gamma": Gamma_FFT,
    "Replicate %gamma": np.round(Approx_gamma, 4), 
    "Gamma %diff": np.round(Gamma_diff, 4),
    "FFT %vega": Vega_FFT,
    "Replicate %vega": np.round(Approx_vega, 4),
    "Vega %diff": np.round(Vega_diff, 4)
    })
print(df)
df.to_csv("Table 3 Panel ABC1.csv")

       S  FFT %delta  Replicate %delta  Delta %diff  FFT %gamma  \
0    950     44.2794           44.2819       0.0057      0.2016   
1    960     46.2918           46.2940       0.0047      0.2008   
2    970     48.2928           48.2945       0.0036      0.1994   
3    980     50.2776           50.2788       0.0023      0.1975   
4    990     52.2414           52.2421       0.0013      0.1952   
5   1000     54.1800           54.1801       0.0003      0.1925   
6   1010     56.0893           56.0890      -0.0006      0.1893   
7   1020     57.9657           57.9649      -0.0014      0.1859   
8   1030     59.8058           59.8046      -0.0021      0.1821   
9   1040     61.6066           61.6049      -0.0027      0.1780   
10  1050     63.3654           63.3633      -0.0032      0.1737   

    Replicate %gamma  Gamma %diff  FFT %vega  Replicate %vega  Vega %diff  
0             0.2016       0.0028    74.9945          74.9679     -0.0354  
1             0.2007      -0.0436    76.250